In [4]:
import os
import vedo
import numpy as np
import trimesh
from copy import copy
from scipy.special import sph_harm
import ipywidgets as widgets
from ipywidgets import interact
import pyvista as pv
from IPython import embed
import re
import pickle as pkl
from utils import appendSpherical_np, cache_base_functions, generate_population
import gif

# Spherical harmonics

The spherical harmonics are defined by the following formula

$$Y_{lm}(\theta, \phi)=\sqrt{\frac{2l+1}{4\pi}}P_l^m(\cos\theta)e^{im\phi},$$

where the $P_l^m$ are the associated Legendre polynomials. They satisfy the following equation:

$$r^2\nabla^2Y_{lm}(\theta, \phi)=l(l+1)Y_{lm}(\theta, \phi)$$

In other words, the Laplacian operator $\nabla^2$ can be diagonalized by using this basis.


### Generative process

1) Sample coefficients $b_{lm}^{(0)}\sim \mathcal{N}(0,\sigma_c^2)$; $a_{lm}^{(n)}, b_{lm}^{(n)}\sim \mathcal{N}(0,\sigma_s^2)$. <br>
2) Build time-dependent coefficients as:
$$a_{lm}(t)=b_{lm}^{(0)}+\sum_{n=1}^{n_{\text{max}}}a_{lm}^{(n)}\sin(2\pi nt)+b_{lm}^{(n)}\cos(2\pi nt),\,\, t\in[0,1]$$<br>
3) The deformation is performed along the radial direction at each point, and with an amplitude given by the following expansion:
$$ \textbf{d}(\textbf{r},t) =\bigg( \sum_{l=0}^{l_{\text{max}}}\sum_{m=-l}^{l}a_{lm}(t)Y_{lm}(\theta, \phi)\bigg) \hat{\textbf{r}}$$

### Ideas

##### A local deformation

The expansion of the Dirac delta is given by:

$$\delta(\theta,\phi)=\sum_{l=0}^{\infty}\sum_{m=-l}^{l}Y_{lm}(0, 0)Y_{lm}(\theta, \phi)$$

Would this be useful to create a local deformation (by truncating this expansion)?

We build a dictionary  $Y_{lm}(\theta, \phi)$ for all the $(\theta, \phi)$ in the discretization of $S_2$. First we get the coordinates of such discretization:

We cache 
$$f_{lmn}(\theta,\phi,t)=Y_{lm}(\theta,\phi)\sin(2\pi nt)$$
$$g_{lmn}(\theta,\phi,t)=Y_{lm}(\theta,\phi)\cos(2\pi nt)$$


### Generate population

In [1]:
import json

In [6]:
params = json.load(open("config.json"))

In [15]:
hash(params.values()) % 1000000

324538

In [ ]:
avg_meshes, moving_meshes, coefs, ref_shape = generate_population(**params)

#### Save population as file

In [ ]:
population = {
  "params": params,
  "time_avg_mesh": avg_meshes,
  "moving_mesh": moving_meshes,
  "coefficients": coefs,
  "template_mesh": ref_shape
  
}

file_pattern = "synthetic_population__N_{N}__T_{T}_" + \
"_sigma_c_{amplitude_static_max}__sigma_s_{amplitude_dynamic_max}_" + \
"_lmax_{l_max}__nmax_{freq_max}__res_{mesh_resolution}__seed_{random_seed}.pkl"
file = file_pattern.format(**params)

with open(file, "wb") as ff:
    pkl.dump(population, ff)

In [ ]:
with open(file, "rb") as ff:
    population = pkl.load(ff)

### Generate GIF animations

In [ ]:
# connectivity
conn = ref_shape.faces
conn = np.c_[3 * np.ones(conn.shape[0]), conn].astype(int)  # add column of 3 to make it PyVista-compatible
gif.generate_gif_population(moving_meshes, mesh_connectivity=conn, show_edges=False)

### Calculate root mean squared deformations

In [ ]:
def rmse(mesh1, mesh2):
    return np.sqrt(((mesh1.vertices - mesh2.vertices)**2).sum(axis=1).mean()) # average over vertices

In [ ]:
rmse_lst = []
for j in range(params["N"]):  
  rmse_j = []
  for i in range(params["T"]):
    rmse_ji = rmse(moving_meshes[j][i], avg_meshes[j])
    rmse_j.append(rmse_ji)
    #print(f"  {rmse_ji:8.4f}")
  print(f"{rmse(ref_shape, avg_meshes[j]):4.4f}, {np.stack(rmse_j).mean():4.4f} ")
  rmse_lst.append(rmse_j)

# Interactive 3D mesh plotting

In [ ]:

sphere = vedo.Sphere(res=params["mesh_resolution"]).to_trimesh()
conn = sphere.faces # connectivity
conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3, as required by PyVista

def f(j, i, average):
    
    if average:
        mesh = pv.PolyData(avg_meshes[j-1].vertices, conn)
    else:
        mesh = pv.PolyData(moving_meshes[j-1][i-1].vertices, conn)
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    pl.add_mesh(mesh, show_edges=False)
    pl.show(use_panel=True, interactive=True, interactive_update=True)
    
#interact(f, i=widgets.IntSlider(min=1,max=N))
interact(f, 
  j=widgets.IntSlider(min=1, max=params["N"]),
  i=widgets.IntSlider(min=1, max=params["T"]),   
  average=widgets.ToggleButton()
);

### Adjust coefficients through sliders

In [ ]:
def apply_deformation(sphere, spherical_coords, amplitude_lm):
    
    deformed_sphere = sphere
    
    coefs = np.ones(spherical_coords.shape[0])
    
    for l, m in amplitude_lm:        
        coefs += amplitude_lm[l, m] * Y_lm[l,m]
                
    deformed_sphere.vertices = np.multiply(
        deformed_sphere.vertices, np.kron(np.ones((3, 1)), coefs).transpose()
    )
    
    return deformed_sphere

def get_coef_as_string(l,m):
    return "a" + str(l) + str(m).replace("-","_")

def get_lm(string):
    import re    
    regex = re.compile(".*([0-9])(_?[0-9])")
    l = int(regex.match(string)[1])
    m = int(regex.match(string)[2].replace("_","-"))
    return l,m    


In [ ]:
from utils import cache_Ylm

In [ ]:
sphere = vedo.Sphere().to_trimesh()
sphere_coords = appendSpherical_np(sphere.vertices)
Y_lm = cache_Ylm(sphere_coords, 2)

In [ ]:
def f(**kwargs):

    coefs = {get_lm(lm):kwargs[lm] for lm in kwargs}

    sphere = vedo.Sphere().to_trimesh()
    conn = np.c_[np.ones(sphere.faces.shape[0]) * 3, sphere.faces].astype(int)  # add column of 3
    spherical_coords = appendSpherical_np(sphere.vertices)[:,3:]
    deformed_sphere = apply_deformation(sphere, spherical_coords, coefs)
        
    mesh = pv.PolyData(deformed_sphere.vertices, conn)
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=True)
    pl.add_mesh(mesh, show_edges=False)
    pl.show(use_panel=True, interactive=True, interactive_update=True)

In [ ]:
indices = [(1,-1), (1,0), (1,1)]

In [ ]:
coefs = {get_coef_as_string(*lm): widgets.FloatSlider(min=-1, max=1) for lm in indices}
interact(f, **coefs);

In [ ]:
# t = 15
# mesh_list[t].show(faces=True)

In [ ]:
#def f(l, m, amplitude):
#    return DeformedSphere().deform(amplitude, l, m).show()

#### Saving the synthetic meshes as `obj` files

In [ ]:
# odir = "/home/rodrigo/tmp/mallas_obj"
# os.makedirs(odir, exist_ok=True)
# 
# N = 100
# 
# for i in range(N):
#     for t, mesh in enumerate(mesh_list):
#         
#         # print(mesh.vertices)
#         j = str(j)
#         j = "0" * (2 - len(j)) + j # 5 --> 05, 15 --> 15
#         
#         with open(os.path.join(odir, "sphere_{}.obj".format(i)), "wt") as ff:        
#             obj_content = trimesh.exchange.export.export_obj(
#                 mesh, 
#                 include_normals=False, 
#                 include_texture=False
#             )
#             ff.write(obj_content)
#             
#         # print(mesh.show())

__________________________

# Attempt to visualize meshes (to complete)

#### `itkwidgets`

In [ ]:
import itk
import itkwidgets
from itkwidgets import view

#### `Trimesh`

In [ ]:
import trimesh

#### `pyvista`

#### `polyscope`

In [ ]:
import polyscope as ps
ps.init()
mesh = mesh_list[25]
ps.register_surface_mesh("sph", mesh.vertices, mesh.faces, smooth_shade=True)
ps.show()

________________________

In [ ]:
DeformedSphere().deform(0, 0, 0).vertices

In [ ]:
interact(f, t=widgets.IntSlider(N_t-1, max=N_t-1));

In [ ]:
# np.polynomial.chebyshev.Chebyshev((1,2,3))()